## Retrieval

- Ý tưởng:
  + Sử dụng 1 model Word2Vec của Google để biến đổi 1 word thành 1 vector có chiều dài 300
  + Với tất cả document trong Dataset thì sẽ áp dụng tiền xử lý rồi dùng model đó để tìm ra vector đại diện cho mỗi câu bằng cách mean của các vector từ
  + Với 1 query document thì cũng áp dụng tiền xử lý, dùng model tìm ra vector đại diện -> Xong thì dùng cosine similarity để trả về các câu có độ tương đồng với câu đầu vào đó

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Chuyển đến vị trí thư mục hiện tại
%cd /content/drive/MyDrive/Colab/chapter6_solution/Bai 6 - Code

/content/drive/.shortcut-targets-by-id/1xiv3cJ70E8QNV0XL9gs6EDLjOnszeQ0O/Bai 6 - Code


In [3]:
!ls

'Chapter 6.ipynb'		      Preprocessing_and_Visualizing.ipynb
 DL_Model.ipynb			      spam.csv
 dts_20k_preprocessed.csv	      weights-improvement-01-7.9131.hdf5
 dts_20k_raw.csv		      weights-improvement-02-7.1496.hdf5
 GoogleNews-vectors-negative300.bin   weights-improvement-03-7.0137.hdf5
 model.png			      weights-improvement-04-6.9062.hdf5
 NaiveBayesClassifier.ipynb	      weights-improvement-05-6.8229.hdf5


In [4]:
!pip install --upgrade --force-reinstall numpy
!pip install --upgrade scipy

  Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.5 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.5 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.5 which is incompatible.
  Using cached scipy-1.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached scipy-1.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (37.6 MB)
  A

In [6]:
!pip install --upgrade --force-reinstall gensim

  Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.1 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached scipy-1.13.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached smart_open-7.1.0-py3-none-any.whl.metadata (24 kB)
  Using cached wrapt-1.17.2-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.4 kB)
Using cached gensim-4.3.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.7 MB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
Using cached scipy-1.13.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (38.6 MB)
Using cached smart_open-7.1.0-py3-none-any.whl (61 kB)
Using cached wrapt-1.17.2-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (83 kB)
  Att

In [4]:
import gensim
from gensim.models import Word2Vec
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import itertools
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import scipy
from scipy import spatial
from nltk.tokenize.toktok import ToktokTokenizer
import re
tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [5]:
# Randomly taking sentences from internet
Doc1 = ["With the Union cabinet approving the amendments to the Motor Vehicles Act, 2016, those caught for drunken driving will have to have really deep pockets, as the fine payable in court has been enhanced to Rs 10,000 for first-time offenders." ]

Doc2 = ["Natural language processing (NLP) is an area of computer science and artificial intelligence concerned with the interactions between computers and human (natural) languages, in particular how to program computers to process and analyze large amounts of natural language data."]

Doc3 = ["He points out that public transport is very good in Mumbai and New Delhi, where there is a good network of suburban and metro rail systems."]

Doc4 = ["But the man behind the wickets at the other end was watching just as keenly. With an affirmative nod from Dhoni, India captain Rohit Sharma promptly asked for a review. Sure enough, the ball would have clipped the top of middle and leg."]

# Put all the documents in one list

all_sentences = Doc1 + Doc2 + Doc3 + Doc4

### Thử nghiệm nlkt.word_tokenize và tokenizer.tokenize của TokTokTokenizer

- nltk.word_tokenize() thường được coi là mạnh hơn và chính xác hơn trong việc xử lý các trường hợp phức tạp so với ToktokTokenizer(). Lý do là vì:

- Mô hình ngữ pháp và quy tắc phức tạp:

  + `nltk.word_tokenize()` sử dụng mô hình Punkt Tokenizer (một mô hình phân tích cú pháp) để tự động nhận diện các từ và dấu câu trong văn bản. Điều này giúp nó xử lý các trường hợp phức tạp như:

    + Tách từ trong các cụm từ ghép.

    + Xử lý dấu câu kề nhau (ví dụ: "Let's" sẽ được tách thành "Let" và "'s").

    + Xử lý các ký tự đặc biệt và các từ lạ trong ngữ cảnh.

  + Xử lý các trường hợp đặc biệt:

    `nltk.word_tokenize()` có thể nhận diện và xử lý các từ hợp nhất (như "I'm", "don't", "let's", v.v.), trong khi ToktokTokenizer có thể gặp khó khăn với các trường hợp này vì nó sử dụng cách tiếp cận đơn giản hơn, chủ yếu dựa vào khoảng trắng và dấu câu.

  + Độ chính xác cao hơn:

    `nltk.word_tokenize()` có thể phân tách chính xác hơn trong các văn bản có dấu câu phức tạp hoặc các cụm từ có dấu câu đặc biệt. Điều này rất quan trọng khi làm việc với văn bản tự nhiên, nơi các ký tự đặc biệt và cấu trúc câu rất đa dạng.

In [9]:
nltk.word_tokenize(Doc1[0])

['With',
 'the',
 'Union',
 'cabinet',
 'approving',
 'the',
 'amendments',
 'to',
 'the',
 'Motor',
 'Vehicles',
 'Act',
 ',',
 '2016',
 ',',
 'those',
 'caught',
 'for',
 'drunken',
 'driving',
 'will',
 'have',
 'to',
 'have',
 'really',
 'deep',
 'pockets',
 ',',
 'as',
 'the',
 'fine',
 'payable',
 'in',
 'court',
 'has',
 'been',
 'enhanced',
 'to',
 'Rs',
 '10,000',
 'for',
 'first-time',
 'offenders',
 '.']

In [10]:
tokenizer.tokenize(Doc1[0])

['With',
 'the',
 'Union',
 'cabinet',
 'approving',
 'the',
 'amendments',
 'to',
 'the',
 'Motor',
 'Vehicles',
 'Act',
 ',',
 '2016',
 ',',
 'those',
 'caught',
 'for',
 'drunken',
 'driving',
 'will',
 'have',
 'to',
 'have',
 'really',
 'deep',
 'pockets',
 ',',
 'as',
 'the',
 'fine',
 'payable',
 'in',
 'court',
 'has',
 'been',
 'enhanced',
 'to',
 'Rs',
 '10,000',
 'for',
 'first-time',
 'offenders',
 '.']

### Load pretrained model Word2Vec

In [11]:
vector_negative_bin_file_path  = "/content/drive/MyDrive/Colab/chapter6_solution/Bai 6 - Code/GoogleNews-vectors-negative300.bin"

In [12]:
# Import the required modules
from gensim.models import KeyedVectors

# Specify the correct function name
model = KeyedVectors.load_word2vec_format(vector_negative_bin_file_path, binary=True)

### Preprocessing

In [14]:
def remove_stopwords(text, is_lower_case=False):
    pattern = r'[^a-zA-z0-9\s]'
    text = re.sub(pattern, '', ''.join(text))
    tokens = nltk.word_tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

### Function to get the embedding vector for n dimension

In [15]:
def get_embedding(word):
    try:
        return model[word]
    except KeyError:
        return np.zeros(model.vector_size)

In [16]:
test_word_embedding = get_embedding("Union")

In [17]:
test_word_embedding.shape

(300,)

Show vector của 1 từ (chiều dài vector là 300 do dùng pretrained có sẵn của Google)

In [18]:
get_embedding("house")

array([ 1.57226562e-01, -7.08007812e-02,  5.39550781e-02, -1.89208984e-02,
        9.17968750e-02,  2.55126953e-02,  7.37304688e-02, -5.68847656e-02,
        1.79687500e-01,  9.27734375e-02,  9.03320312e-02, -4.12109375e-01,
       -8.30078125e-02, -1.45507812e-01, -2.37304688e-01, -3.68652344e-02,
        8.74023438e-02, -2.77099609e-02,  1.13677979e-03,  8.30078125e-02,
        3.57421875e-01, -2.61718750e-01,  7.47070312e-02, -8.10546875e-02,
       -2.35595703e-02, -1.61132812e-01, -4.78515625e-02,  1.85546875e-01,
       -3.97949219e-02, -1.58203125e-01, -4.37011719e-02, -1.11328125e-01,
       -1.05957031e-01,  9.86328125e-02, -8.34960938e-02, -1.27929688e-01,
       -1.39648438e-01, -1.86523438e-01, -5.71289062e-02, -1.17675781e-01,
       -1.32812500e-01,  1.55639648e-02,  1.34765625e-01,  8.39843750e-02,
       -9.03320312e-02, -4.12597656e-02, -2.51953125e-01, -2.27539062e-01,
       -6.64062500e-02, -7.66601562e-02,  5.15136719e-02,  5.90820312e-02,
        3.49609375e-01, -

### Tính vector cho câu bằng cách lấy trung bình của các embedding từ các từ trong câu.

In [38]:
out_dict =  {}
for sentence in all_sentences:
    average_vector = (np.mean(np.array([get_embedding(x) for x in nltk.word_tokenize(remove_stopwords(sentence.lower()))]), axis=0))
    dict = { sentence : (average_vector) }
    out_dict.update(dict)

In [39]:
out_dict

{'With the Union cabinet approving the amendments to the Motor Vehicles Act, 2016, those caught for drunken driving will have to have really deep pockets, as the fine payable in court has been enhanced to Rs 10,000 for first-time offenders.': array([-0.00752952,  0.07640769,  0.02021651,  0.0393538 , -0.06667813,
        -0.09641058,  0.0038473 , -0.08695915,  0.06166562,  0.05776978,
        -0.01717099, -0.07339061, -0.01688454,  0.02896396, -0.14146839,
         0.03586648,  0.08766868,  0.04445579,  0.03018492, -0.03743397,
         0.07872425, -0.0311307 , -0.0437896 , -0.02006392,  0.09745927,
        -0.05626887, -0.08198131,  0.06809304,  0.03912354, -0.03560014,
        -0.01045366, -0.06907515,  0.01930514,  0.08539789,  0.02101829,
        -0.08427767,  0.02317116, -0.00695523, -0.01170072,  0.05518185,
         0.04779608, -0.03513406,  0.13249796, -0.02304086, -0.09093406,
        -0.0787298 ,  0.00950484,  0.02489818, -0.0392567 ,  0.01530179,
        -0.03363592,  0.0462

### Function to calculate the similarity between the **query vector** and **document vector**


In [40]:
def get_similarity(query_embedding, average_vector_doc):
    sim = [(1 - scipy.spatial.distance.cosine(query_embedding, average_vector_doc))]
    return sim

In [58]:
# Rank all the documents based on the similarity to get relevant docs
def ranked_documents(query):
    #Tính vector trung bình cho chuỗi truy vấn
    query_words = np.mean(np.array([get_embedding(x) for x in nltk.word_tokenize(remove_stopwords(query.lower()))],dtype=float), axis=0)
    rank = []
    for k,v in out_dict.items():
        rank.append((k, get_similarity(query_words, v)))
    rank = sorted(rank,key=lambda t: t[1], reverse=True)
    print('Ranked Documents :')
    return rank

In [50]:
get_similarity([1,2,3], [1,2,3])

[1.0]

In [67]:
ranked_documents("With the Union cabinet approving")

Ranked Documents :


[('With the Union cabinet approving the amendments to the Motor Vehicles Act, 2016, those caught for drunken driving will have to have really deep pockets, as the fine payable in court has been enhanced to Rs 10,000 for first-time offenders.',
  [0.5257626816961933]),
 ('He points out that public transport is very good in Mumbai and New Delhi, where there is a good network of suburban and metro rail systems.',
  [0.27723365002179334]),
 ('But the man behind the wickets at the other end was watching just as keenly. With an affirmative nod from Dhoni, India captain Rohit Sharma promptly asked for a review. Sure enough, the ball would have clipped the top of middle and leg.',
  [0.21373832056834186]),
 ('Natural language processing (NLP) is an area of computer science and artificial intelligence concerned with the interactions between computers and human (natural) languages, in particular how to program computers to process and analyze large amounts of natural language data.',
  [0.175454

In [64]:
ranked_documents('With the Union cabinet approving the amendments to the Motor Vehicles Act, 2016, those caught for drunken driving will have to have really deep pockets')

Ranked Documents :


[('With the Union cabinet approving the amendments to the Motor Vehicles Act, 2016, those caught for drunken driving will have to have really deep pockets, as the fine payable in court has been enhanced to Rs 10,000 for first-time offenders.',
  [0.8812577269377971]),
 ('But the man behind the wickets at the other end was watching just as keenly. With an affirmative nod from Dhoni, India captain Rohit Sharma promptly asked for a review. Sure enough, the ball would have clipped the top of middle and leg.',
  [0.47518011636993696]),
 ('He points out that public transport is very good in Mumbai and New Delhi, where there is a good network of suburban and metro rail systems.',
  [0.4419815397236223]),
 ('Natural language processing (NLP) is an area of computer science and artificial intelligence concerned with the interactions between computers and human (natural) languages, in particular how to program computers to process and analyze large amounts of natural language data.',
  [0.3736062

In [53]:
ranked_documents(Doc1[0])

Ranked Documents :


[('With the Union cabinet approving the amendments to the Motor Vehicles Act, 2016, those caught for drunken driving will have to have really deep pockets, as the fine payable in court has been enhanced to Rs 10,000 for first-time offenders.',
  [1.0]),
 ('But the man behind the wickets at the other end was watching just as keenly. With an affirmative nod from Dhoni, India captain Rohit Sharma promptly asked for a review. Sure enough, the ball would have clipped the top of middle and leg.',
  [0.5105212145035003]),
 ('He points out that public transport is very good in Mumbai and New Delhi, where there is a good network of suburban and metro rail systems.',
  [0.48402211451969834]),
 ('Natural language processing (NLP) is an area of computer science and artificial intelligence concerned with the interactions between computers and human (natural) languages, in particular how to program computers to process and analyze large amounts of natural language data.',
  [0.39162706165865124])]

### Cải tiến

In [ ]:
!pip install annoy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 27.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp311-cp311-linux_x86_64.whl size=553317 sha256=28c42283aca1123d3713b8e8bf08cdf9d27a0ad06eba58b60640469f7a09b3f5
  Stored in directory: /root/.cache/pip/wheels/33/e5/58/0a3e34b92bedf09b4c57e37a63ff395ade6f6c1099ba59877c
Successfully built annoy


In [ ]:
import numpy as np
import nltk
import scipy.spatial.distance
from annoy import AnnoyIndex
import pickle # Để lưu map index <-> câu

# Giả sử bạn đã có hàm get_embedding và hàm remove_stopwords
# Giả sử all_sentences là list các câu của bạn
# Giả sử EMBEDDING_DIM là kích thước của vector embedding

print("Building document vectors...")
out_dict = {}
doc_vectors = []
doc_indices = [] # Lưu thứ tự gốc của câu

for i, sentence in enumerate(all_sentences):
    tokens = nltk.word_tokenize(remove_stopwords(sentence.lower()))
    if not tokens:
        continue

    # Tính vector trung bình, đảm bảo không bị lỗi nếu chỉ có 1 token
    sentence_vector = np.mean(np.array([get_embedding(x) for x in tokens]), axis=0)
    # Kiểm tra xem vector có hợp lệ không (tránh NaN nếu get_embedding lỗi)
    if not np.isnan(sentence_vector).any():
        out_dict[sentence] = sentence_vector
        doc_vectors.append(sentence_vector)
        doc_indices.append(i) # Lưu index gốc

Building document vectors...


In [ ]:
out_dict['With the Union cabinet approving the amendments to the Motor Vehicles Act, 2016, those caught for drunken driving will have to have really deep pockets, as the fine payable in court has been enhanced to Rs 10,000 for first-time offenders.'].shape

(300,)

In [ ]:
# --- Xây dựng Annoy Index ---
print("Building Annoy index...")
embedding_dim = 300 # Thay bằng kích thước thật của embedding
annoy_index = AnnoyIndex(embedding_dim, 'angular') # 'angular' tương ứng với cosine distance

Building Annoy index...


In [ ]:
save_path_index = '/content/mydocuments.ann'
save_path_map = '/content/index_map.pkl'

In [ ]:
# Tạo map từ index của Annoy về câu gốc
index_to_sentence_map = {}
for annoy_idx, original_idx in enumerate(doc_indices):
    sentence = all_sentences[original_idx]
    vector = out_dict[sentence]
    annoy_index.add_item(annoy_idx, vector)
    index_to_sentence_map[annoy_idx] = sentence

# Xây dựng index - số cây (n_trees) càng lớn càng chính xác nhưng chậm hơn
# và tốn bộ nhớ hơn. Cần thử nghiệm để chọn giá trị phù hợp.
annoy_index.build(100)
print(f"Saving index to: {save_path_index}")
annoy_index.save(save_path_index)

Saving index to: /content/mydocuments.ann


True

In [ ]:
def ranked_documents_annoy(query, num_results=10):
    # Tính vector trung bình cho chuỗi truy vấn
    query_tokens = nltk.word_tokenize(query.lower())
    if not query_tokens:
         print("Query is empty or contains only stopwords.")
         return []
    # Lấy embedding, đảm bảo dtype=float để tránh lỗi mean với object
    query_embeddings = [get_embedding(x) for x in query_tokens if get_embedding(x) is not None] # Bỏ qua từ không có embedding
    if not query_embeddings:
        print("No valid embeddings found for query tokens.")
        return []
    query_vector = np.mean(np.array(query_embeddings, dtype=float), axis=0)

    if np.isnan(query_vector).any():
         print("Could not compute a valid query vector.")
         return []

    # Tìm kiếm các hàng xóm gần nhất bằng Annoy
    # get_nns_by_vector trả về list các index của Annoy
    # include_distances=True trả về cả khoảng cách (không phải similarity)
    neighbor_indices, distances = annoy_index.get_nns_by_vector(
        query_vector,
        num_results,
        include_distances=True
    )

    # Lấy lại câu gốc và tính similarity (nếu cần chính xác)
    # Hoặc chỉ cần sắp xếp theo distance trả về từ Annoy (distance nhỏ -> similarity cao)
    results = []
    for annoy_idx, dist in zip(neighbor_indices, distances):
        sentence = index_to_sentence_map[annoy_idx]
        # Annoy 'angular' distance d = sqrt(2*(1-cos(u,v)))
        # Nên cos(u,v) = 1 - (d^2)/2
        similarity = 1 - (dist**2) / 2
        results.append((sentence, similarity))

    # Sắp xếp lại theo similarity giảm dần (vì Annoy trả về theo distance tăng dần)
    # results.sort(key=lambda t: t[1], reverse=True) # Thường không cần vì Annoy đã sắp xếp theo distance

    print(f"Top {len(results)} relevant documents found:")
    return results

In [ ]:
my_query = "cabinet"
top_docs = ranked_documents_annoy(my_query, num_results=5)
for doc, sim in top_docs:
    print(f"- Similarity: {sim:.4f} | Document: {doc}")

Top 4 relevant documents found:
- Similarity: 0.3570 | Document: With the Union cabinet approving the amendments to the Motor Vehicles Act, 2016, those caught for drunken driving will have to have really deep pockets, as the fine payable in court has been enhanced to Rs 10,000 for first-time offenders.
- Similarity: 0.2153 | Document: He points out that public transport is very good in Mumbai and New Delhi, where there is a good network of suburban and metro rail systems.
- Similarity: 0.1817 | Document: But the man behind the wickets at the other end was watching just as keenly. With an affirmative nod from Dhoni, India captain Rohit Sharma promptly asked for a review. Sure enough, the ball would have clipped the top of middle and leg.
- Similarity: 0.1237 | Document: Natural language processing (NLP) is an area of computer science and artificial intelligence concerned with the interactions between computers and human (natural) languages, in particular how to program computers to proc